# Notebook 1: Homology matrix generation from genome sequences

In this notebook, I will be applying the notebooks accompanying the paper by Norsigian et al., 2020. (doi:10.1038/s41596-019-0254-3.) I will apply this for the P. thermo model we've been working on and the M10EXG strain that we used to validate our model with.

This is the the first notebook in the tutorial to create homology matrix from genome sequences.There are four major steps in this notebook
1. Download the genome annotation (GenBank files) from NCBI, and generate fasta files (protein &nucleotide) from them
2. Perform BLASTp to find homologous proteins in strains of interest
3. Use best bidirectional hits to create gene presence/absence matrix
4. Supplementary for best practice: use BLASTn to check if we have missed any unannotated open reading frames and retain these genes in orthology matrix as well as guide future manual curation

In [1]:
#import packages needed
import pandas as pd
from glob import glob

In [2]:
from Bio import Entrez, SeqIO

In [3]:
import sys

In [4]:
import cobra

In [5]:
import decimal

__NOTE__ to be able to import the Entrez and SeqIO, I need to change the folder name from 'bio' to 'Bio' and then it'll work. 

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\Lib\site-packages

So be careful whenever i install Biopython again that this needs to be fixed.

Here I will be working with strains in the faculative anaerobic clade of the genus. I will also add genomes that are obligate aerobes to see if that could highlight to us what changed between these species that made them become obligate aerobes. 

In [7]:
# Load the information on the five strains we will be working with in this tutorial
StrainsOfInterest=pd.read_excel('Strain Information.xlsx')
StrainsOfInterest

,Strain,NCBI ID,Physiology
0,Goebacillus thermoglucosidasius M10EXG,2501416905,Facultative anaerobe


In [8]:
#The Reference Genome is as Described in the Base Reconstruction; here the reference is 
referenceStrainID='NCIMB11955'
targetStrainIDs=list(StrainsOfInterest['NCBI ID'])

## 1. Download genome annotations (GenBank files) to generate fasta files 

### Dowload genomes from NCBI
Download the genome annotations (GenBank files) from NCBI for strains of interest. 

In [9]:
# define a function to download the annotated genebank files from NCBI
def dl_genome(id, folder='genomes'): # be sure get CORRECT ID
    files=glob('%s/*.gb'%folder)
    out_file = '%s/%s.gb'%(folder, id)

    if out_file in files:
        print (out_file, 'already downloaded')
        return
    else:
        print ('downloading %s from NCBI'%id)
        
    from Bio import Entrez
    Entrez.email = "vivmol@biosustain.dtu.dk"     #Insert email here for NCBI
    handle = Entrez.efetch(db="nucleotide", id=id, rettype="gb", retmode="text")
    fout = open(out_file,'w')
    fout.write(handle.read())
    fout.close()

In [24]:
# execute the above function, and download the GenBank files for 8 P. thermo strains
for strain in targetStrainIDs:
    dl_genome(strain, folder='genomes')

downloading CP016552.1 from NCBI
downloading CP020030.1 from NCBI
downloading CP012712.1 from NCBI
downloading CP002835.1 from NCBI
downloading CP001638 from NCBI
downloading CP008903.1 from NCBI


In [25]:
#also download the reference strain info
dl_genome(referenceStrainID, folder='genomes')

downloading CP016622.1 from NCBI


### Examine the Downloaded Strains

In [10]:
# define a function to gather information of the downloaded strains from the GenBank files
def get_strain_info(folder='genomes'):
    files = glob('%s/*.gb'%folder)
    strain_info = []
    
    for file in files:
        handle = open(file)
        record = SeqIO.read(handle, "genbank")
        
        for f in record.features:
            if f.type=='source':
                info = {}
                info['file'] = file
                info['id'] = file.split('\\')[-1].split('.')[0]
                for q in f.qualifiers.keys():
                    info[q] = '|'.join(f.qualifiers[q])
                strain_info.append(info)
    return pd.DataFrame(strain_info)

In [10]:
# information on the downloaded strain
get_strain_info(folder='genomes')

""


### Generate FASTA files for both Protein and Nucleotide Pipelines
From the GenBank file, we can extract sequence and annoation information to generate fasta files for the protein and nucleotide analyses. The resulting fasta files will then be used in step 2 as input for BLAST 

In [11]:
# define a function to parse the Genbank file to generate fasta files for both protein and nucleotide sequences
def parse_genome(id, type='prot', in_folder='genomes', out_folder='prots', overwrite=1):

    in_file = '%s/%s.gb'%(in_folder, id)
    out_file='%s/%s.fa'%(out_folder, id)
    files =glob('%s/*.fa'%out_folder)
    
    if out_file in files and overwrite==0:
        print (out_file, 'already parsed')
        return
    else:
        print ('parsing %s'%id)
    
    handle = open(in_file)
    
    fout = open(out_file,'w')
    x = 0
    
    records = SeqIO.parse(handle, "genbank")
    for record in records:
        for f in record.features:
            if f.type=='CDS':
                seq=f.extract(record.seq)
                
                if type=='nucl':
                    seq=str(seq)
                else:
                    seq=str(seq.translate())
                    
                if 'locus_tag' in f.qualifiers.keys():
                    locus = f.qualifiers['locus_tag'][0]
                elif 'gene' in f.qualifiers.keys():
                    locus = f.qualifiers['gene'][0]
                else:
                    locus = 'gene_%i'%x
                    x+=1
                fout.write('>%s\n%s\n'%(locus, seq))
    fout.close()

In [12]:
# Generate fasta files for 5 strains of interest
for strain in targetStrainIDs:
    parse_genome(strain, type='prot', in_folder='genomes', out_folder='prots')
    parse_genome(strain, type='nucl', in_folder='genomes', out_folder='nucl')


parsing 2501416905
parsing 2501416905


In [13]:
#Also generate fasta files for the reference strain
parse_genome(referenceStrainID, type='nucl', in_folder='genomes', out_folder='nucl')
parse_genome(referenceStrainID, type='prots', in_folder='genomes', out_folder='prots')

parsing NCIMB11955
parsing NCIMB11955


## 2. Perform BLAST to find homologous proteins in strains of interest

### Make BLAST DB for each of the target strains for both Protein and Nucleotide Pipelines

In this tutorial, we will run both BLASTp for proteins and BLSATn for nucleotides. BLASTp will be used as the main approach to identify homologous proteins in reference strain and other strains of interest, while BLASTn will be used as a supplementary method to check for any unannotated genes

In [14]:
# Define a function to make blast database for either protein of nucleotide
def make_blast_db(id,folder='prots',db_type='prot'):
    import os
    
    out_file ='%s/%s.fa.pin'%(folder, id)
    files =glob('%s/*.fa.pin'%folder)
    
    if out_file in files:
        print (id, 'already has a blast db')
        return
    if db_type=='nucl':
        ext='fna'
    else:
        ext='fa'

    cmd_line='makeblastdb -in %s/%s.%s -dbtype %s' %(folder, id, ext, db_type)
    
    print ('making blast db with following command line...')
    print (cmd_line)
    os.system(cmd_line)

In [15]:
sys.path.append('..\\..\\..\\..\\..\\..\\Program Files\\NCBI\\blast-2.10.1+\\bin')

In [16]:
# make protein sequence databases 
# Because we are performing bi-directional blast, we make databases from both reference strain and strains of interest
for strain in targetStrainIDs:
    make_blast_db(strain,folder='prots',db_type='prot')
make_blast_db(referenceStrainID,folder='prots',db_type='prot')

making blast db with following command line...
makeblastdb -in prots/2501416905.fa -dbtype prot
making blast db with following command line...
makeblastdb -in prots/NCIMB11955.fa -dbtype prot


### Define functions to run protein BLAST and get sequence lengths
- BLASTp will be the main approach used here to identify homologous proteins between strains 
- Aside from sequence similarity, we also want to ensure the coverage of sequence mapping is sufficient. Therefore, we need to identiy the sequence length for each protein and compare it with the alignment length.

In [17]:
# define a function to run BLASTp
def run_blastp(seq,db,in_folder='prots', out_folder='bbh', out=None,outfmt=6,evalue=0.001,threads=1):
    import os
    if out==None:
        out='%s/%s_vs_%s.txt'%(out_folder, seq, db)
        print(out)
    
    files =glob('%s/*.txt'%out_folder)
    if out in files:
        print (seq, 'already blasted')
        return
    
    print ('blasting %s vs %s'%(seq, db))
    
    db = '%s/%s.fa'%(in_folder, db)
    seq = '%s/%s.fa'%(in_folder, seq)
    cmd_line='blastp -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastp with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [18]:
# define a function to get sequence length 

def get_gene_lens(query, in_folder='prots'):

    file = '%s/%s.fa'%(in_folder, query)
    handle = open(file)
    records = SeqIO.parse(handle, "fasta")
    out = []
    
    for record in records:
        out.append({'gene':record.name, 'gene_length':len(record.seq)})
    
    out = pd.DataFrame(out)
    return out

## 3. Use Bi-Directional BLASTp Best Hits to create gene presence/absence matrix

### Obtain Bi-Directional BLASTp Best Hits

From the above BLASTp results, we can obtain Bi-Directional BLASTp Best Hits to identify homologous proteins. Note beside gene similarity score, the coverage of alignment is also used to filter mapping results. 

In [19]:
# define a function to get Bi-Directional BLASTp Best Hits
def get_bbh(query, subject, in_folder='bbh'):    
    
    #Utilize the defined protein BLAST function
    run_blastp(query, subject)
    run_blastp(subject, query)
    
    query_lengths = get_gene_lens(query, in_folder='prots')
    subject_lengths = get_gene_lens(subject, in_folder='prots')
    
    #Define the output file of this BLAST
    out_file = '%s/%s_vs_%s_parsed.csv'%(in_folder,query, subject)
    files=glob('%s/*_parsed.csv'%in_folder)
    
    #Combine the results of the protein BLAST into a dataframe
    print ('parsing BBHs for', query, subject)
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    bbh=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,query, subject), sep='\t', names=cols)
    bbh = pd.merge(bbh, query_lengths) 
    bbh['COV'] = bbh['alnLength']/bbh['gene_length']
    
    bbh2=pd.read_csv('%s/%s_vs_%s.txt'%(in_folder,subject, query), sep='\t', names=cols)
    bbh2 = pd.merge(bbh2, subject_lengths) 
    bbh2['COV'] = bbh2['alnLength']/bbh2['gene_length']
    out = pd.DataFrame()
    
    # Filter the genes based on coverage
    bbh = bbh[bbh.COV>=0.25]
    bbh2 = bbh2[bbh2.COV>=0.25]
    
    #Delineate the best hits from the BLAST
    for g in bbh.gene.unique():
        res = bbh[bbh.gene==g]
        if len(res)==0:
            continue
        best_hit = res.loc[res.PID.idxmax()]
        best_gene = best_hit.subject
        res2 = bbh2[bbh2.gene==best_gene]
        if len(res2)==0:
            continue
        best_hit2 = res2.loc[res2.PID.idxmax()]
        best_gene2 = best_hit2.subject
        if g==best_gene2:
            best_hit['BBH'] = '<=>'
        else:
            best_hit['BBH'] = '->'
        out=pd.concat([out, pd.DataFrame(best_hit).transpose()])
    
    #Save the final file to a designated CSV file    
    out.to_csv(out_file)

In [20]:
# Execute the BLAST for each target strain against the reference strain, save results to 'bbh' i.e. "bidirectional best
# hits" folder to create
# homology matrix

for strain in targetStrainIDs:
    get_bbh(referenceStrainID,strain, in_folder='bbh')

bbh/NCIMB11955_vs_2501416905.txt
blasting NCIMB11955 vs 2501416905
running blastp with following command line...
blastp -db prots/2501416905.fa -query prots/NCIMB11955.fa -out bbh/NCIMB11955_vs_2501416905.txt -evalue 0.001 -outfmt 6 -num_threads 1
bbh/2501416905_vs_NCIMB11955.txt
blasting 2501416905 vs NCIMB11955
running blastp with following command line...
blastp -db prots/NCIMB11955.fa -query prots/2501416905.fa -out bbh/2501416905_vs_NCIMB11955.txt -evalue 0.001 -outfmt 6 -num_threads 1
parsing BBHs for NCIMB11955 2501416905


C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\ipykernel_launcher.py:46 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\pandas\core\indexing.py:671 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\ipykernel_launcher.py:44 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

### Parse the BLAST Results into one Homology Matrix of the Reconstruction Genes

For the homology matrix, want to find, for each gene in the reference annotation, is there one in the other strains. And then later filter this down to metabolic genes. 

In [21]:
#Load all the BLAST files between the reference strain and target strains

blast_files=glob('%s/*_parsed.csv'%'bbh')

for blast in blast_files:
    bbh=pd.read_csv(blast)
    print (blast,bbh.shape) 

bbh\NCIMB11955_vs_2501416905_parsed.csv (3520, 16)


In this section of the notebook, I will deviate from the published tutorial. In the tutorial, they map the orthologous genes onto the curated model of the reference genome. In reality, we are curious as to how homologous the genomes are to one another, and how many metabolic genes the different strains have in common. So we need to compare the orthologues to the reference genome and not the reference model. I'll try to adapt the scripts so that we can get a homology matrix that captures this.

Make a single dataframe where one column is all the genes in the reference organism, and the other column is a different strain and contains the PID.

Then you can count for each column, the number of genes which have a PID above 80% (selected threshold) and compare it to the total number of genes.

In [22]:
#import all the csv files
compare = pd.read_csv('bbh/NCIMB11955_vs_2501416905_parsed.csv')

In [23]:
#filter out all other columns that i won't use later
compare = compare[['gene', 'PID']]

In [377]:
#list of all ORFs found in the reference genome
with open('prots/NCIMB11955.fa') as fasta_file:  # Will close handle cleanly
    NCIMB_ids = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        NCIMB_ids.append(seq_record.id)

In [288]:
comparison = pd.DataFrame({'gene':  NCIMB_ids})

In [289]:
comparison = pd.merge(comparison, compare, on='gene',how="outer")

In [290]:
strains = ['NCIMB11955', 'M10EXG']

In [28]:
comparison.columns = strains

Now that we have a dataframe that compares the PID scores for each gene in the reference genome to the other strains, we can start to 'sum up' what percentage of the genes in the reference have a matched gene in the strain. 

For this, we will set a threshold of 80% sequence identity between genes to be counted as a true homologue. One can decide to change this arbitrarily set value if they like. 

The reference genome has 3708 ORFs annotated to it. 

In [29]:
columns = list(comparison) 
  
for i in columns: #iterate through the columns
    if i in 'NCIMB11955': # skip reference column
        continue
    else:
        #now go through each row in this column
        common_genes = []
        for index,row in comparison.iterrows():
            value = row[i]
            if value > 90: #selected threshold level
                common_genes.append(1)
            elif value < 90:
                common_genes.append(0)
    homology = sum(common_genes)
    fraction = 100*homology/3708
    print(i,': ', fraction)            

M10EXG :  88.9967637540453


Try the same as above, but use the E-value as the threshold instead.

In [30]:
#import all the csv files
compare = pd.read_csv('bbh/NCIMB11955_vs_2501416905_parsed.csv')

In [31]:
#filter out all other columns that i won't use later
compare_eval = compare[['gene', 'eVal']]

In [32]:
#make dataframe for first comparison, and then add on the rest
comparison_eval = pd.DataFrame({'gene':  NCIMB_ids})

In [33]:
comparison_eval = pd.merge(comparison_eval, compare_eval, on='gene',how="outer")

In [34]:
strains = ['NCIMB11955', 'M10EXG']

In [35]:
comparison_eval.columns = strains

In [36]:
columns = list(comparison_eval) 
  
for i in columns: #iterate through the columns
    if i in 'NCIMB11955': # skip reference column
        continue
    else:
        #now go through each row in this column
        common_genes = []
        for index,row in comparison_eval.iterrows():
            value = row[i]
            if value < 5E-5 : #selected threshold level
                common_genes.append(1)
            elif value >5E-5:
                continue
    homology = sum(common_genes)
    fraction = (homology/3708)*100
    print(i,': ', fraction)
                

M10EXG :  94.47141316073355


Now that we have done the above, we can start to look at the overlap between the strains, so that we can create a venn diagram of total ORFs that are unique to each strain but also ovelap.


__Approach__
- Import the fasta files in the prots folder: this is a list of all the ORFs in each strain. 

- make a list, per strain, of all genes for that strain that fit the comparison threshold (for this use Eval < 1E-10 for now) i.e. all the genes that these two strains have in common

- then make an overlap of the all the lists: should give for each strain which are unique and which overlap with one another.


In [225]:
#import total strain lists, for each strain
with open('prots/2501416905.fa') as fasta_file:  # Will close handle cleanly
    M10EXG_ids = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        M10EXG_ids.append(seq_record.id)

Next is to make a dataframe for the comparison of the reference (here NCIMB11955) to the other strain. The list should then contain the gene names of the reference organism, as well as the strain being mapped against. 

In [38]:
compare_eval

,gene,eVal
0,gene_0,0.000000e+00
1,gene_1,2.600000e-41
2,gene_2,2.500000e-104
3,gene_3,1.620000e-54
4,gene_4,7.660000e-127
...,...,...
3515,gene_3752,3.860000e-76
3516,gene_3753,0.000000e+00
3517,gene_3754,6.070000e-51
3518,gene_3755,4.680000e-87


In [39]:
ol_reference = [] #the overlapping genes, with the reference strain ID
ol_strain =[] #the overlapping genes, with the reference strain ID
for index,row in compare.iterrows():
    value = row['eVal']
    if value > 1E-5: #selected threshold level
        continue #we don't want to save these anywhere            
    elif value < 1E-5:
        ol_reference.append(row['gene'])
        ol_strain.append(row['subject'])
NCIMB_M10EXG_OL =  pd.DataFrame({'NCIMB11955': ol_reference, 'M10EXG': ol_strain})

In [40]:
# so NCIMB_M10EXG_OL is now a dataframe that maps each gene in the NCIMB to a gene in M10EXG

In [41]:
len(NCIMB_M10EXG_OL)

3498

In the example above, you then see which 3498 genes of the target strain match the reference strain.
We can then see how many genes each strain has by themselves and from that make the venndiagram.

In [42]:
len(NCIMB_ids)

3757

So NCIMB has 259 unique ORFs. 

In the above, we had NCIMB as the reference strain. TO find how many true unique genes there are in M10EXG, we would need to repeat the above analysis but now with M10EXG as reference strain. That will be done below.

In [203]:
StrainsOfInterest=pd.read_excel('Strain InformationB.xlsx')
StrainsOfInterest

,Strain,NCBI ID,Physiology
0,Goebacillus thermoglucosidasius NCIMB11955,NCIMB11955,Facultative anaerobe


In [204]:
#switch reference and target here
referenceStrainID='2501416905'
targetStrainIDs=list(StrainsOfInterest['NCBI ID'])

In [205]:
for strain in targetStrainIDs:
    get_bbh(referenceStrainID,strain, in_folder='bbh')

bbh/2501416905_vs_NCIMB11955.txt
blasting 2501416905 vs NCIMB11955
running blastp with following command line...
blastp -db prots/NCIMB11955.fa -query prots/2501416905.fa -out bbh/2501416905_vs_NCIMB11955.txt -evalue 0.001 -outfmt 6 -num_threads 1
bbh/NCIMB11955_vs_2501416905.txt
blasting NCIMB11955 vs 2501416905
running blastp with following command line...
blastp -db prots/2501416905.fa -query prots/NCIMB11955.fa -out bbh/NCIMB11955_vs_2501416905.txt -evalue 0.001 -outfmt 6 -num_threads 1
parsing BBHs for 2501416905 NCIMB11955


C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\ipykernel_launcher.py:44 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\pandas\core\indexing.py:671 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\ipykernel_launcher.py:46 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

Now import this file and make it into a dataframe so we can find the unique genes.

In [213]:
#import all the csv files
compare = pd.read_csv('bbh/2501416905_vs_NCIMB11955_parsed.csv')

In [219]:
#filter out all other columns that i won't use later
compare_eval = compare[['gene', 'eVal', 'subject']]

In [222]:
strains = ['M10EXG', 'eVal', 'NCIMB11955']

In [223]:
compare_eval.columns = strains

Now that we have done the above, we can slook at the overlap from M10EXg to the NCIMB strain. 


__Approach__
- make a list, per strain, of all genes for that strain that fit the comparison threshold (for this use Eval < 1E-5 for now) i.e. all the genes that these two strains have in common


In [231]:
ol_reference = [] #the overlapping genes, with the reference strain ID
ol_strain =[] #the overlapping genes, with the reference strain ID
for index,row in compare_eval.iterrows():
    value = row['eVal']
    if value > 1E-5: #selected threshold level
        continue #we don't want to save these anywhere            
    elif value < 1E-5:
        ol_reference.append(row['M10EXG'])
        ol_strain.append(row['NCIMB11955'])
M10EXG_NCIMB_OL =  pd.DataFrame({'M10EXG': ol_reference, 'NCIMB': ol_strain})

In [40]:
# so M10EXG_NCIMB_OL is now a dataframe that maps each gene in the M10EXG to a gene in NCIMB11955

In [233]:
len(M10EXG_NCIMB_OL)

3493

There are 5 genes less mapped as overlap, but this is within the error range.

In [234]:
len(M10EXG_ids)

3727

So we would have 234 unique genes in M10EXG. 

So overall, we have 3498 genes that overlap, 259 unique in NCIMB and 234 unique in M10EXG. This very closely matches the results of the KBASE pipeline we did, which is good.

## Filter for metabolic genes
Now that we know the overlap in the total protein, we want to find out what percentage of the metabolic genes overlap between the strains. So first I will filter all the genes from the target organism into the ones that are metabolic genes. And then apply this list to the list of bi-directional hits, so see how many of the metabolic genes overlap. 

I can't get the definition to work, So I will just write a script that will make a dataframe linking each gene to a possible EC code. Then we can filter the bbh-file made previously for these metabolic genes and get the answer that way.

In [328]:
genes = []
EC = []
x = 0
for seq_record in SeqIO.parse("genomes/NCIMB11955.gb", "genbank"):
     for f in seq_record.features:
        if f.type=='CDS':
            if 'locus_tag' in f.qualifiers.keys():
                locus = f.qualifiers['locus_tag'][0]
            elif 'gene' in f.qualifiers.keys():
                locus = f.qualifiers['gene'][0]
            else:
                locus = 'gene_%i'%x
                x+=1
            try:
                synonyms = f.qualifiers['gene_synonym']
                #here it will check that it has one of the gene_synonyms as ec code, i.e. is metabolic
                check = []
                for a in synonyms:
                    if a[0].isdigit():
                        ec = a
                        check.append(1)
                    else:
                        continue
                if sum(check) > 0:
                    
                    genes.append(locus)
                    EC.append(a)
            except KeyError:
                continue


In [329]:
NCIMB_met = pd.DataFrame({'Gene': genes, 'EC': EC})

Now I'll do the same for the M10EXG genome/annotation.

In [237]:
genes = []
EC = []
x = 0
for seq_record in SeqIO.parse("genomes/2501416905.gb", "genbank"):
     for f in seq_record.features:
        if f.type=='CDS':
            if 'locus_tag' in f.qualifiers.keys():
                locus = f.qualifiers['locus_tag'][0]
            elif 'gene' in f.qualifiers.keys():
                locus = f.qualifiers['gene'][0]
            else:
                locus = 'gene_%i'%x
                x+=1
            try:
                synonyms = f.qualifiers['gene_synonym']
                #here it will check that it has one of the gene_synonyms as ec code, i.e. is metabolic
                check = []
                for a in synonyms:
                    if a[0].isdigit():
                        ec = a
                        check.append(1)
                    else:
                        continue
                if sum(check) > 0:
                    
                    genes.append(locus)
                    EC.append(a)
            except KeyError:
                continue


In [238]:
M10EXG_met = pd.DataFrame({'Gene': genes, 'EC': EC})

In [239]:
len(NCIMB_met)

1424

In [240]:
len(M10EXG_met)

1417

So, this shows we have 1424 metabolic genes in the reference strain and 1417 in the M10EXG strain. Note, there are ofcourse many hypothetical proteins annotated in the genome. Those are ignored. 
Now, I will have to filter the list of BBH based on just the genes that are metabolic. 

Approach:
- NCIMB_M10EXG_OL has all the CDS that are considered hits here (with the threshold of 1E-5 set). I will iterate through each row, and make a new dataframe which contains the filtered set of genes for comparing.

NOTE: at the end of comparing with NCIMC11955 as reference, I will do the same with the M10EXG as reference, to find which genes are unique there as well.

In [241]:
NCIMB = []
M10EXG = []
for row, index in NCIMB_M10EXG_OL.iterrows():
    ref_gene = index['NCIMB11955']
    try:
        NCIMB_met.loc[NCIMB_met["Gene"] == ref_gene,'EC'].values[0] #if it is a metaoblic gene this will give a hit
        NCIMB.append(index['NCIMB11955'])
        M10EXG.append(index['M10EXG'])
    except IndexError: #if the hit isn't metabolic, we can ignore it
        continue

In [242]:
#make data frame
OL_metabolic = pd.DataFrame({'NCIMB11955':NCIMB, 'M10EXG':M10EXG})

In [243]:
len(OL_metabolic)

1384

So we have 1384 metabolic genes that overlap. That means there are 40 genes unique to NCIMB, and for M10EXG we need to check how many are still unmatched.

Finally, I will make a list of the unique metabolic genes, that can be supplied in supplementary, and given a short look through if anything unexpected appears.

Approach:
- NCIMB_met and M10EXG_met has all the metabolic genes with their corresponding EC code. I will filter out the significant hits here for the NCIMB strain first. I need to repeat the above analysis for M10EXG to be able to see how many unique it has properly. (will be done further in this notebook)
- then try to find some more information about each E.C. code, e.g. from KEGG database I've Used before? I can look into the type of reactions (i.e. category) and/or the extact reaction name.

In [244]:
genes = []
ECs =[]
for row, index in NCIMB_met.iterrows():
    gene = index['Gene']
    try:
        OL_metabolic.loc[OL_metabolic["NCIMB11955"] == gene,'M10EXG'].values[0] #If the gene is in the overlap dataframe it will give an output
        continue
    except IndexError: #i.e. if the gene doesn't have an overlap in M10EXG
        genes.append(gene)
        ECs.append(index['EC'])
        
#make dataframe
NCIMB_unique = pd.DataFrame({'Unique gene':genes, 'EC':ECs}) 

In [245]:
len(NCIMB_unique)

40

Now we know that for NCIMB, there are 40 unique metabolic genes and 1384 overlapping genes. I will do the same as above but for the M10EXG strain to find how many are unique in that strain when that is used as reference sequence.

In [248]:
M10EXG = []
NCIMB = []
for row, index in M10EXG_NCIMB_OL.iterrows():
    ref_gene = index['M10EXG']
    try:
        M10EXG_met.loc[M10EXG_met["Gene"] == ref_gene,'EC'].values[0] #if it is a metaoblic gene this will give a hit
        M10EXG.append(index['M10EXG'])
        NCIMB.append(index['NCIMB'])
    except IndexError: #if the hit isn't metabolic, we can ignore it
        continue

In [252]:
#make data frame
OL_metabolic_M10EXG = pd.DataFrame({'M10EXG':M10EXG, 'NCIMB11955':NCIMB})

In [253]:
len(OL_metabolic_M10EXG)

1388

Again here there is a difference of a few genes, but falls within an error range if you consider the size of the total metabolic gene set. So this is fine. Next, we will make a dataframe of the metabolic genes that are unique to M10EXG.

In [254]:
genes = []
ECs =[]
for row, index in M10EXG_met.iterrows():
    gene = index['Gene']
    try:
        OL_metabolic_M10EXG.loc[OL_metabolic_M10EXG["M10EXG"] == gene,'NCIMB11955'].values[0] #If the gene is in the overlap dataframe it will give an output
        continue
    except IndexError: #i.e. if the gene doesn't have an overlap in M10EXG
        genes.append(gene)
        ECs.append(index['EC'])
        
#make dataframe
M10EXG_unique = pd.DataFrame({'Unique gene':genes, 'EC':ECs})

In [255]:
len(M10EXG_unique)

29

So we have 29 unique genes in M10EXG, and 40 in our strain. 

Now we have two dataframes, one for each strain, with the unique metabolic genes, we can try to find a bit more information about them. I will try to get a name for the reaction, as well as what pathway they are a part of. 

First I will prepare a dataframe from the KEGG Site that contains the information about which pathway the EC code belongs to.

In [256]:
df = pd.read_csv('http://rest.kegg.jp/link/ec/pathway', header=None, sep = '\t')

In [257]:
df.columns = ['Pathway', 'EC'] #rename the columns

In [258]:
#remove all 'path:' and 'rn:'
df['Pathway'] = df['Pathway'].str.replace(r'path:ec', '')

In [259]:
df['EC'] = df['EC'].str.replace(r'ec:', '')

In [260]:
#remove the rows with 'path_map' to prevent duplication
df = df[~df['Pathway'].str.contains("map")]

In [261]:
#now link the pathway code to the name of the pathway it is involved in 

In [262]:
df_groups = pd.read_csv('http://rest.kegg.jp/list/pathway', header=None, sep = '\t')

In [263]:
df_groups.columns = ['Pathway', 'Name']

In [264]:
df_groups['Pathway'] = df_groups['Pathway'].str.replace(r'path:map', '')

In [265]:
#now filter out the IDs I dont want to include
#i want to remove all rows below number 153
df_groups = df_groups[0:154]

In [266]:
#then merge the df and df_groups together
df = pd.merge(df_groups,df,on='Pathway',how='left')

Now I can map the EC code from the unique metabolic reaction lists to these pathway classes.

In [267]:
pathway =[]
for index, row in NCIMB_unique.iterrows():
    ec = row['EC']
    types =[]
    found = df.loc[df["EC"] == ec]
    for indexa, rowa in found.iterrows() :
        types.append(rowa['Name']) 
    pathway.append(types)
NCIMB_unique['Pathway'] = pathway

In [268]:
pathway =[]
for index, row in M10EXG_unique.iterrows():
    ec = row['EC']
    types =[]
    found = df.loc[df["EC"] == ec]
    for indexa, rowa in found.iterrows() :
        types.append(rowa['Name']) 
    pathway.append(types)
M10EXG_unique['Pathway'] = pathway

Now I'll also add a column which looks at the KO of the ec codes recognized so that we get a bit more information about which reactions are unique in each strain.

First I will prepare a dataframe that contains the EC codes linked to the KO ontology terms for these annotations. That we can use to map the unique reactions to.
There will probably still be some that are not mapped, those we will need to check by hand.

In [269]:
df = pd.read_csv('http://rest.kegg.jp/link/ec/ko', header=None, sep = '\t')

In [270]:
df.columns = ['KO', 'EC'] #rename the columns

In [271]:
#remove all 'ko:' and 'ec:'
df['KO'] = df['KO'].str.replace(r'ko:', '')

In [272]:
df['EC'] = df['EC'].str.replace(r'ec:', '')

In [273]:
#now import the list of KO terms with more meaningful description
ko = pd.read_csv('http://rest.kegg.jp/list/ko', header=None, sep = '\t')

In [274]:
ko.columns = ['KO', 'Name']

In [275]:
ko['KO'] = ko['KO'].str.replace(r'ko:', '')

In [276]:
#link the two dataframes together
df = pd.merge(df,ko,on='KO',how='left')

Now we can map the EC code to a KO term.

In [277]:
ko_term =[]
for index, row in NCIMB_unique.iterrows():
    ec = row['EC']
    types =[]
    found = df.loc[df["EC"] == ec]
    for indexa, rowa in found.iterrows() :
        types.append(rowa['Name']) 
    ko_term.append(types)
NCIMB_unique['KO'] = ko_term

In [278]:
ko_term =[]
for index, row in M10EXG_unique.iterrows():
    ec = row['EC']
    types =[]
    found = df.loc[df["EC"] == ec]
    for indexa, rowa in found.iterrows() :
        types.append(rowa['Name']) 
    ko_term.append(types)
M10EXG_unique['KO'] = ko_term

Now I will export these two tables and need to some some manual inspection of them.

In [279]:
M10EXG_unique.to_csv('M10EXG_unique.csv')

In [280]:
NCIMB_unique.to_csv('NCIMB_unique.csv')